In [10]:
import pandas as pd
import numpy as np
output_pickle = '/project/projectdirs/metatlas/projects/jgi_projects/20170222_omalley_fungus/pactolus_hits.pkl'
pactolus_df = pd.read_pickle(output_pickle)
pactolus_df.reset_index(inplace=True)
peak_df = pd.read_csv('/global/u2/b/bpb/Downloads/OMalley_peaks.txt',sep='\t')
peak_df.head()

,polarity,mz,rt,comment
0,positive,357.0378,2.77,not in control not in rumen
1,positive,566.5631,7.05,cluster1
2,positive,580.5778,7.20,cluster1
3,positive,594.5938,7.35,cluster1
4,positive,608.6091,7.48,cluster1


In [16]:
def get_matching_msms(row,msms_df=None,mz_tolerance=10,rt_tolerance=0.2):
    """
    
    """
    hits = np.where((abs(msms_df['precursor_mz'].values - row.mz) / row.mz * 1.0e6 < mz_tolerance) & 
                    ( abs(msms_df['retention_time'].values - row.rt) < rt_tolerance  ))
    return hits
hits = peak_df.apply(get_matching_msms,msms_df=pactolus_df,axis=1)
hits = hits.apply(lambda x: x[0])
df = pd.DataFrame({'scores':hits})
peak_to_msms_df = pd.DataFrame([[i, x] 
                   for i, y in df.scores.apply(list).iteritems() 
                   for x in y], columns=['peak_index','msms_index']).set_index('peak_index')

peak_to_msms_df.reset_index(inplace=True)


labeled_df = peak_df.merge(peak_to_msms_df,how='outer',left_index=True,right_on='peak_index')
temp = labeled_df.merge(pactolus_df,how='left',left_on='msms_index',right_index=True)
temp.to_csv('/global/u2/b/bpb/Downloads/omalley_select_peak_pactolus.csv')